In [18]:
import numpy
from numpy import *

In [19]:
def loadDataSet(fileName):
    dataMat=[];labelMat=[]
    fr=open(fileName)
    for line in fr.readlines():
        lineArr=line.strip().split('\t')
        dataMat.append([float(lineArr[0]),float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

In [20]:
def selectJrand(i,m):# i is the index of our first alpha,m is the total number of alphas
    j=i
    while(j==i):#A value is randomly chosen and returned as long as it's not equal to the input i
        j=int(random.uniform(0,m))
    return j

In [21]:
def clipAlpha(aj,H,L):#clips alpha values that are greater than H or less than L
    if aj>H:
        aj=H
    if L>aj:
        aj=L
    return aj

In [28]:
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):#the dataset, the class labels, a constant C, the tolerance,the maximum number of iterations
    dataMatrix=mat(dataMatIn);labelMat=mat(classLabels).transpose()#This makes the row of the class labels correspond to the row of the data matrix
    b=0;m,n=shape(dataMatrix)
    alphas=mat(zeros((m,1)))
    iter=0
    while(iter<maxIter):
        alphaPairsChanged=0#record if the attempt to optimize any alphas worked
        for i in range(m):
            fXi=float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b#this is our prediction of the class
            Ei=fXi-float(labelMat[i])
            #Alphas will be clipped at 0 or C, so if they’re equal to these, they’re “bound” and can’t be increased or decreased, so it’s not worth trying to optimize these alphas.
            if ((labelMat[i]*Ei<-toler) and (alphas[i]<C)) or ((labelMat[i]*Ei>toler) and (alphas[i]>0)):##若alphas[i]的误差很大,则对该数据实例对应的alpha值进行优化  
                j=selectJrand(i,m)#选取与i不同的alpha值  
                #计算样本j的预测类别 
                fXj=float(multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T))+b
                Ej=fXj-float(labelMat[j])
                #由于python是通过引用的方式传递列表,故复制alphai\j以备后面需要alpha旧值
                alphaIold=alphas[i].copy()
                alphaJold=alphas[j].copy()
                #计算L和H,用于将alpha值调整到0-C之间 
                if labelMat[i]!=labelMat[j]:
                    L=max(0,alphas[j]-alphas[i])
                    H=min(C,C+alphas[j]-alphas[i])
                else:
                    L=max(0,alphas[j]+alphas[i]-C)
                    H=min(C,alphas[j]+alphas[i])
                if L==H:
                    print "L==H";continue
                #Eta is the optimal amount to change alpha[j]
                eta=2.0*dataMatrix[i,:]*dataMatrix[j,:].T-dataMatrix[i,:]*dataMatrix[i,:].T-dataMatrix[j,:]*dataMatrix[j,:].T
                if eta>=0:
                    print "eta>=0";continue
                alphas[j]-=labelMat[j]*(Ei-Ej)/eta
                alphas[j]=clipAlpha(alphas[j],H,L)
                if(abs(alphas[j]-alphaJold)<0.00001):
                    print "j not moving enough";continue
                alphas[i]+=labelMat[j]*labelMat[i]*(alphaJold-alphas[j])
                #set the constant term b for these two alphas
                b1=b-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T-labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T
                b2=b-Ej-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T-labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T
                if 0<alphas[i] and C>alphas[i]:
                    b=b1
                elif 0<alphas[j] and C>alphas[j]:
                    b=b2
                else:
                    b=(b1+b2)/2.0
                alphaPairsChanged+=1
                print "iter: %d i: %d, pairs changed %d" %(iter,i,alphaPairsChanged)
            #check to see if any alphas have been updated; if so you set iter to 0 and continue
            if alphaPairsChanged==0:
                iter+=1
            else:
                iter=0
            print "iteration number:%d"%iter
        return b,alphas

In [29]:
dataArr,labelArr=loadDataSet('testSet.txt')